In [1]:
!kubecolor -n argocd get all 

NAME                                                    READY   STATUS    RESTARTS   AGE
pod/argocd-application-controller-0                     1/1     Running   0          15m
pod/argocd-applicationset-controller-66c9b84df4-9gllr   1/1     Running   0          15m
pod/argocd-dex-server-5df4cc4565-8hxwr                  1/1     Running   0          15m
pod/argocd-notifications-controller-6f54fd9856-c8gr6    1/1     Running   2          2d1h
pod/argocd-redis-75ddd9dfbd-kngqp                       1/1     Running   0          38m
pod/argocd-repo-server-86448f84fb-5m9d8                 1/1     Running   0          15m
pod/argocd-server-8549d6695c-gdsnx                      1/1     Running   0          15m

NAME                                       TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
service/argocd-applicationset-controller   ClusterIP   10.97.224.172    <none>        7000/TCP            5d
service/argocd-dex-server                  ClusterIP   10.107.84.11

In [2]:
!kubecolor -n argocd get services

NAME                               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
argocd-applicationset-controller   ClusterIP   10.97.224.172    <none>        7000/TCP            5d
argocd-dex-server                  ClusterIP   10.107.84.113    <none>        5556/TCP,5557/TCP   5d
argocd-redis                       ClusterIP   10.103.145.36    <none>        6379/TCP            5d
argocd-repo-server                 ClusterIP   10.102.103.207   <none>        8081/TCP            5d
argocd-server                      ClusterIP   10.103.146.1     <none>        80/TCP,443/TCP      5d


In [3]:
!nmap -p 8081 argocd-repo-server.argocd.svc

Starting Nmap 7.80 ( https://nmap.org ) at 2024-02-12 08:04 -03
Nmap scan report for argocd-repo-server.argocd.svc (10.102.103.207)
Host is up (0.00026s latency).
rDNS record for 10.102.103.207: argocd-repo-server.argocd.svc.cluster2.xpt

PORT     STATE SERVICE
8081/tcp open  blackice-icecap

Nmap done: 1 IP address (1 host up) scanned in 0.13 seconds


# Ingress

In [2]:
!kubectl -n argocd get ingress argocd-server -o yaml | yq

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    meta.helm.sh/release-name: argocd
    meta.helm.sh/release-namespace: argocd
  creationTimestamp: "2024-01-29T09:41:25Z"
  generation: 1
  labels:
    app.kubernetes.io/component: server
    app.kubernetes.io/instance: argocd
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: argocd-server
    app.kubernetes.io/part-of: argocd
    app.kubernetes.io/version: v2.9.5
    helm.sh/chart: argo-cd-5.53.3
  name: argocd-server
  namespace: argocd
  resourceVersion: "671333"
  uid: 17a45eec-3882-4d54-a7c9-e78f3821336a
spec:
  ingressClassName: nginx
  rules:
    - host: argocd.worldl.xpt
      http:
        paths:
          - backend:
              service:
                name: argocd-server
                port:
                  number: 80
            path: /
            pathType: Prefix
status:
  loadBalancer:
    ingress:
      - ip: 192.168.49.2


In [4]:
! password=$(kubectl -n argocd get secret argocd-initial-admin-secret -o=jsonpath='{.data.password}' | base64 -d) && \
  argocd login argocd.worldl.xpt --grpc-web --username admin --password $password

'admin:login' logged in successfully
Context 'argocd.worldl.xpt' updated


In [7]:
!argocd app list

NAME  CLUSTER  NAMESPACE  PROJECT  STATUS  HEALTH  SYNCPOLICY  CONDITIONS  REPO  PATH  TARGET


In [8]:
!argocd proj list

NAME          DESCRIPTION   DESTINATIONS  SOURCES  CLUSTER-RESOURCE-WHITELIST  NAMESPACE-RESOURCE-BLACKLIST  SIGNATURE-KEYS  ORPHANED-RESOURCES
default                     *,*           *        */*                         <none>                        <none>          disabled
labtools-k8s  labtools-k8s  <none>        <none>   <none>                      <none>                        <none>          disabled


In [13]:
!argocd proj get labtools-k8s -o yaml | yq

metadata:
  creationTimestamp: "2024-02-12T11:53:16Z"
  generation: 1
  managedFields:
    - apiVersion: argoproj.io/v1alpha1
      fieldsType: FieldsV1
      fieldsV1:
        f:spec:
          .: {}
          f:description: {}
        f:status: {}
      manager: argocd-server
      operation: Update
      time: "2024-02-12T11:53:16Z"
  name: labtools-k8s
  namespace: argocd
  resourceVersion: "821515"
  uid: 7a7dbeb2-2adf-4e25-9314-2cdb759c2f84
spec:
  description: labtools-k8s
status: {}


In [6]:
!kubectl -n argocd get serviceaccount argocd-server -o yaml | yq

apiVersion: v1
automountServiceAccountToken: true
kind: ServiceAccount
metadata:
  annotations:
    meta.helm.sh/release-name: argocd
    meta.helm.sh/release-namespace: argocd
  creationTimestamp: "2024-02-07T10:54:21Z"
  labels:
    app.kubernetes.io/component: server
    app.kubernetes.io/instance: argocd
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: argocd-server
    app.kubernetes.io/part-of: argocd
    app.kubernetes.io/version: v2.10.0
    helm.sh/chart: argo-cd-5.55.0
  name: argocd-server
  namespace: argocd
  resourceVersion: "8471"
  uid: cc9e2b9c-7a79-4c51-965c-a95b1f84ed79


In [15]:
!kubectl -n commander get ClusterRole | grep -i argocd

argocd-application-controller                                          2024-02-07T10:54:21Z
argocd-notifications-controller                                        2024-02-07T10:54:21Z
argocd-repo-server                                                     2024-02-07T10:54:21Z
argocd-server                                                          2024-02-07T10:54:21Z


In [16]:
!kubectl -n commander get ClusterRoleBinding | grep -i argocd

argocd-application-controller                          ClusterRole/argocd-application-controller                                          5d2h
argocd-notifications-controller                        ClusterRole/argocd-notifications-controller                                        5d2h
argocd-repo-server                                     ClusterRole/argocd-repo-server                                                     5d2h
argocd-server                                          ClusterRole/argocd-server                                                          5d2h


In [17]:
!kubectl get ClusterRole argocd-repo-server -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  annotations:
    meta.helm.sh/release-name: argocd
    meta.helm.sh/release-namespace: argocd
  creationTimestamp: "2024-02-07T10:54:21Z"
  labels:
    app.kubernetes.io/component: repo-server
    app.kubernetes.io/instance: argocd
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: argocd-repo-server
    app.kubernetes.io/part-of: argocd
    app.kubernetes.io/version: v2.10.0
    helm.sh/chart: argo-cd-5.55.0
  name: argocd-repo-server
  resourceVersion: "8492"
  uid: 93b76429-e701-48f6-94d7-f898c335864b
rules:
  - apiGroups:
      - '*'
    resources:
      - '*'
    verbs:
      - '*'
  - nonResourceURLs:
      - '*'
    verbs:
      - '*'


In [18]:
!kubectl get ClusterRoleBinding argocd-repo-server -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  annotations:
    meta.helm.sh/release-name: argocd
    meta.helm.sh/release-namespace: argocd
  creationTimestamp: "2024-02-07T10:54:21Z"
  labels:
    app.kubernetes.io/component: repo-server
    app.kubernetes.io/instance: argocd
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: argocd-repo-server
    app.kubernetes.io/part-of: argocd
    app.kubernetes.io/version: v2.10.0
    helm.sh/chart: argo-cd-5.55.0
  name: argocd-repo-server
  resourceVersion: "8498"
  uid: b357b1b5-3cba-447d-bf80-f9376cfe5d2c
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: argocd-repo-server
subjects:
  - kind: ServiceAccount
    name: argocd-repo-server
    namespace: argocd
